In [116]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import nltk
from nltk.corpus import brown
from nltk import FreqDist
from nltk.corpus import stopwords
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.cross_validation import cross_val_score
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 9)
plt.style.use('ggplot')

In [69]:
#Importing complete and labled job summary data
complete_data = pd.read_csv('complete_labeled_job_summary_data.csv',encoding = "ISO-8859-1")
complete_data = complete_data.drop('Unnamed: 0',axis=1)

#converting text data into lower case letters in order to process better
complete_data["job summary"]=complete_data["job summary"].str.lower()

In [70]:
#Splitting the data into test and train set and resetting their index
train, test = train_test_split(complete_data, test_size = 0.2)
train=train.reset_index(drop=True)
test=test.reset_index(drop=True)

In [71]:
#Converting strings in train data into nltk.text.Text format in order to use NLP

#Creating empty list for iterations
iterator = []

#Creating a list with length equal to number of rows in the dataframe
for i in range (0,len(train.axes[0]),1):
    iterator.append(i)

#Converting and replacing the strings with nltk.text    
for i in iterator:
    job_summary=train.get_value(i,'job summary')  # Pull just the job summary as a str
    tokens = nltk.word_tokenize(job_summary)      # Convert str to token
    text_obj = nltk.text.Text(tokens)             # convert tokens to nltk.text.Text()
    train.set_value(i,'job summary',text_obj)     # Now all of the functions from lab 17 will work with this obj
    

#Converting strings in test data into nltk.text.Text format in order to use NLP

#Creating empty list for iterations
iterator = []

#Creating a list with length equal to number of rows in the dataframe
for i in range (0,len(test.axes[0]),1):
    iterator.append(i)

#Converting and replacing the strings with nltk.text    
for i in iterator:
    job_summary=test.get_value(i,'job summary')  # Pull just the job summary as a str
    tokens = nltk.word_tokenize(job_summary)      # Convert str to token
    text_obj = nltk.text.Text(tokens)             # convert tokens to nltk.text.Text()
    test.set_value(i,'job summary',text_obj)     # Now all of the functions from lab 17 will work with this obj

In [110]:
#exploring train set
print(train.info())
print('\n',train.describe())
print('\n',train['label'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7294 entries, 0 to 7293
Data columns (total 3 columns):
job title      7294 non-null object
job summary    7294 non-null object
label          7294 non-null object
dtypes: object(3)
memory usage: 85.5+ KB
None

        job title                                        job summary    label
count       7294                                               7294     7294
unique      4250                                               7294        5
top         Host  (accounting, /, human, resources, coordinatorg...  science
freq         275                                                  1     1931

 science     1931
commerce    1855
arts        1372
services    1134
sports      1002
Name: label, dtype: int64


In [143]:
#Creating dummy variables for catagories
train["commerce"] = train["label"].map({'commerce':1, 'science':0, 'arts':0, 'services':0, 'sports':0})
train["science"] = train["label"].map({'commerce':0, 'science':1, 'arts':0, 'services':0, 'sports':0})
train["arts"] = train["label"].map({'commerce':0, 'science':0, 'arts':1, 'services':0, 'sports':0})
train["services"] = train["label"].map({'commerce':0, 'science':0, 'arts':0, 'services':1, 'sports':0})
train.describe()

,commerce,science,arts,services
count,7294.000000,7294.000000,7294.000000,7294.000000
mean,0.254319,0.264738,0.188100,0.155470
std,0.435507,0.441224,0.390819,0.362377
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000


In [ ]:
#Creating a combined list of all the words in job summary

#Creating empty list for iterations and other for words in job summary
iterator = []

#Creating a list with length equal to number of rows in the dataframe
for i in range (0,len(train.axes[0]),1):
    iterator.append(i)
    
#creating the list
for i in iterator:
    j